In [0]:
# !pip install -q tensorflow==2.0.0-alpha0

In [2]:
!git clone https://github.com/aiddun/binary-mnist.git

Cloning into 'binary-mnist'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36
Unpacking objects: 100% (36/36), done.
Checking out files: 100% (50/50), done.


In [0]:
import tensorflow as tf
# from tensorflow.keras.layers import Input,Conv2D,Flatten,Dense,BatchNormalization,Activation,LeakyReLU,Reshape,Conv2DTranspose
# from tensorflow.layers import Input,Conv2D,Flatten,Dense,BatchNormalization,Activation,LeakyReLU,Reshape,Conv2DTranspose

import matplotlib.pyplot as plt
import sys,os
import numpy as np



In [0]:
##################### MOUNT YOUT GOOGLE DRIVE AS A FOLDER ######################
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [0]:
def log_sum_exp(x, axis=None):
    '''Numerically stable log( sum( exp(A) ) ).
    '''
    x_max = tf.reduce_max(x, axis=axis, keepdims=True)
    y=tf.exp(x - x_max)
#     print(y.shape)
    y=tf.reduce_sum(y, axis=axis, keepdims=True)
#     print(y.shape)
    y = tf.log(y) + x_max
#     print(y.shape)
#     y = tf.reduce_sum(y, axis=axis, keepdims=True)
#     print(y.shape)
    return y

def datagen_norepeat(batch_size, data):
    num_all=len(data)
    arr=np.arange(0,num_all)
    np.random.shuffle(arr)
    start=0
    while True:
        if start>=num_all or start+batch_size>num_all:
            start=0
            np.random.shuffle(arr)
            continue
        end=start+batch_size
        yield np.copy(arr[start:end])
        start=start+batch_size
        
def setup_out_dir(out_path):
    if out_path is None:
        raise ValueError('Please set `--out_path` (`-o`) argument.')
    
    binary_dir = os.path.join(out_path, 'binaries')
    image_dir = os.path.join(out_path, 'images')
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
        os.mkdir(binary_dir)
        os.mkdir(image_dir)
    return binary_dir, image_dir

def load_dataset(source):
    print("Reading MNIST, ")
    
    if not os.path.exists(source):
        raise ValueError('Please set source directory')
    data=np.load(source)
    print("Total number of data", data.shape[0])
    idx=np.arange(data.shape[0])
    np.random.shuffle(idx)
    data = np.reshape(data[idx[0:40000]], (-1,28,28,1))
    print("Sample number of data", data.shape[0])
    return data

In [0]:
class Bsgan_binary():
    def __init__(self):
        
        pass
    
    def build(self, latent_dim=100, img_shape=(28,28,1),batch_size=64):
        self.latent_dim=latent_dim
        self.img_shape=img_shape
        self.batch_size=batch_size
        
    
    def build_generator(self, noise, reuse=None, training=True):
        dimx,dimy,dimz=self.img_shape
        with tf.variable_scope("gen", reuse=reuse):
            
#             x=tf.layers.dense(noise, units=7*7*16)
# #             x=tf.layers.batch_normalization(x,training=training)
#             x=tf.keras.layers.BatchNormalization()(x)
#             x=tf.nn.leaky_relu(x,alpha=0.1)
#             x=tf.reshape(x, (-1,7,7,16))
#             x=tf.layers.conv2d_transpose(x, filters=8, kernel_size=5, strides=2, padding="same",activation=None)
# #             x=tf.layers.batch_normalization(x,training=training)
#             x=tf.keras.layers.BatchNormalization()(x)
#             x=tf.nn.leaky_relu(x,alpha=0.1)
#             x=tf.layers.conv2d_transpose(x, filters=dimz, kernel_size=5, strides=2, padding="same", activation=None)
#             m_prob=tf.nn.sigmoid(x)
            
            # big model
            x=tf.layers.dense(noise, units=1024)
#             x=tf.layers.batch_normalization(x,training=training)
            x=tf.keras.layers.BatchNormalization()(x)
            x=tf.layers.dense(x, units=128*7*7)
#             x=tf.layers.batch_normalization(x,training=training)
            x=tf.keras.layers.BatchNormalization()(x)
            x=tf.reshape(x, (-1,7,7,128))
            x=tf.layers.conv2d_transpose(x, filters=64, kernel_size=5, strides=2, padding="same",activation=None)
#             x=tf.layers.batch_normalization(x,training=training)
            x=tf.keras.layers.BatchNormalization()(x)
            x=tf.layers.conv2d_transpose(x, filters=1, kernel_size=5, strides=2, padding="same",activation=None)
            m_prob=tf.nn.sigmoid(x)
            
            print("Generator output shape: ",m_prob.shape)

            return m_prob
        
    def build_discriminator(self, im, reuse=None,training=True):
        dimx,dimy,dimz=self.img_shape
        with tf.variable_scope("disc", reuse=reuse):
            
#             x=tf.reshape(im, shape=(-1,dimx,dimy,dimz))
#             x=tf.layers.conv2d(x, filters=8, kernel_size=5, strides=2, padding="same",activation=None) # 16 filters
#             x=tf.nn.leaky_relu(x,alpha=0.1)
#             x=tf.layers.conv2d(x, filters=16, kernel_size=5, strides=2, padding="same",activation=None)# 32 filters
# #             x=tf.layers.batch_normalization(x,training=training)
#             x=tf.keras.layers.BatchNormalization()(x)
#             x=tf.nn.leaky_relu(x,alpha=0.1)
#             x=tf.layers.Flatten()(x)
#             m_logit=tf.layers.dense(x, units=1, activation=None) # logit is used for loss calculation
            
            # Big model
            x=tf.layers.conv2d(im, filters=64, kernel_size=5, strides=2, padding="same",activation=None)
            x=tf.nn.leaky_relu(x,alpha=0.2)
            
            x=tf.layers.conv2d(x, filters=128, kernel_size=5, strides=2, padding="same",activation=None)
            x=tf.nn.leaky_relu(x,alpha=0.2)
            x=tf.contrib.layers.flatten(x)
            x=tf.layers.dense(x, units=1024, activation=None)
            x=tf.nn.leaky_relu(x,alpha=0.2)
            m_logit=tf.layers.dense(x, units=1, activation=None)
            
            print("Discriminator output shape: ",m_logit.shape)

        return m_logit

    def lrelu(self,x,alpha=0.2):
        return tf.nn.leaky_relu(x,alpha)
        
    def loss_fn(self,g_out,real_out, is_training):
        
        dimx,dimy,dimz=self.img_shape
        n_samples=self.n_samples
        batch_size=self.batch_size

        
        R=tf.random.uniform(shape=(n_samples, batch_size,dimx,dimy,dimz),dtype=tf.float32)
        sample=tf.cast(R<=tf.expand_dims(g_out, axis=0),dtype=tf.float32)
        fake_out=self.build_discriminator(
            tf.reshape(sample, (n_samples*batch_size,dimx,dimy,dimz)),reuse=True,training=False)#d output for fake samples
        fake_out_=tf.reshape(fake_out, (n_samples, batch_size))
        
        log_w=fake_out_
        
        g_out_=tf.clip_by_value(g_out, 1e-7, 1.-1e-7)
        log_g=tf.reduce_sum(sample * tf.log(g_out_) + (1. - sample)*tf.log(1. - g_out_), axis=[2,3,4], keepdims=False)
        
        log_N=tf.log(tf.cast(log_w.shape[0],dtype=tf.float32))
        log_Z_est=log_sum_exp(log_w - log_N, axis=0)
        log_w_tilde = log_w - log_Z_est - log_N
        w_tilde = tf.exp(log_w_tilde)
        w_tilde_ = tf.stop_gradient(w_tilde)
        
        print("ccc",log_g.shape, w_tilde_.shape, log_Z_est.shape, (w_tilde_*log_g).shape)
        g_loss= - tf.reduce_mean(tf.reduce_sum(w_tilde_*log_g, axis=0))
        d_loss= tf.reduce_mean(tf.nn.softplus(-real_out))+tf.reduce_mean(tf.nn.softplus(-fake_out)) + tf.reduce_mean(fake_out)
        
        return g_loss, d_loss, real_out, fake_out, log_w, log_g
    
    def train(self,source, binary_dir, image_dir,stepoch=0, retrain=True,n_samples=20,epochs=20000, sample_interval=50, lr_g=1e-4, lr_d=1e-4):
        
        self.n_samples=n_samples
        self.lr_g=lr_g
        self.lr_d=lr_d
        
        # load dataset,create batch
        x_train=load_dataset(source).astype("float32")
        dg=datagen_norepeat(self.batch_size, x_train)
        
        
        if retrain:
            # Build graph
            tf.reset_default_graph()
            im=tf.placeholder(dtype=tf.float32,shape=(None,28,28,1), name="im")
            noise=tf.placeholder(dtype=tf.float32,shape=(None,self.latent_dim), name="noise")
            is_training = tf.placeholder(dtype=tf.bool, name='is_training')
            self.generator=self.build_generator(noise, training=is_training)
            self.discriminator = self.build_discriminator(im, training=is_training)

            g_loss, d_loss, real_out, fake_out,log_w, log_g=self.loss_fn(self.generator,self.discriminator, is_training)
            
            # Name tensors
            g_loss=tf.identity(g_loss, name="g_loss")
            d_loss=tf.identity(d_loss, name="d_loss")
            real_out=tf.identity(real_out, name="real_out")
            fake_out=tf.identity(fake_out, name="fake_out")
            log_w=tf.identity(log_w, name="log_w")
            log_g=tf.identity(log_g, name="log_g")
            self.generator=tf.identity(self.generator, name="g_out")
            self.discriminator=tf.identity(self.discriminator, name="d_out")
            
            
            #
            var_g=[var for var in tf.trainable_variables() if var.name.startswith("gen")]
            var_d=[var for var in tf.trainable_variables() if var.name.startswith("disc")]
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                optimizer_d = tf.train.RMSPropOptimizer(learning_rate=lr_d, name="op_d").minimize(d_loss, var_list=var_d)
                optimizer_g = tf.train.RMSPropOptimizer(learning_rate=lr_g, name="op_g").minimize(g_loss, var_list=var_g)
            
            sess = tf.Session()
            sess.run(tf.global_variables_initializer())
            

        else:
            sess = tf.Session()
            loader = tf.train.import_meta_graph(binary_dir+"/lrg{}_lrd{}.meta".format(self.lr_g, self.lr_d))
            loader.restore(sess, tf.train.latest_checkpoint(binary_dir))
            graph=tf.get_default_graph()
            im = graph.get_tensor_by_name("im:0")
            noise=graph.get_tensor_by_name("noise:0")
            is_training=graph.get_tensor_by_name("is_training:0")
            d_loss=graph.get_tensor_by_name("d_loss:0")
            g_loss=graph.get_tensor_by_name("g_loss:0")
            real_out=graph.get_tensor_by_name("real_out:0")
            fake_out=graph.get_tensor_by_name("fake_out:0")
            log_w=graph.get_tensor_by_name("log_w:0")
            log_g=graph.get_tensor_by_name("log_g:0")
            self.generator=graph.get_tensor_by_name("g_out:0")
            self.discriminator=graph.get_tensor_by_name("d_out:0")
            optimizer_d=graph.get_operation_by_name("op_d")
            optimizer_g=graph.get_operation_by_name("op_g")
            
        saver=tf.train.Saver() # create saver
        
        # Results dictionary
        self.res_dic={"g_loss":np.zeros((epochs,)), # create results saver
                "d_loss":np.zeros((epochs,)),
                "p_real":np.zeros((epochs,)),
                "p_fake":np.zeros((epochs,)),
                "log_w":np.zeros((epochs,)),
                "log_g":np.zeros((epochs,))}
        
        counter=0
        train_d=True
        for epoch in range(stepoch,epochs):
            
            idx=next(dg)
            real=x_train[idx]
            
            n = np.random.normal(0.0, 1.0, (self.batch_size, self.latent_dim)).astype("float32")
#             if train_d:
#                 sess.run(optimizer_d, feed_dict={noise:n, im:real, is_training:True})
#                 counter+=1
#                 if counter==2:
#                     counter=0
#                     train_d=False
#             else:
#                 sess.run(optimizer_g, feed_dict={noise:n, is_training:True})
#                 train_d=True
            
            n = np.random.normal(0.0, 1.0, (self.batch_size, self.latent_dim)).astype("float32")
            if train_d: # train d
                sess.run(optimizer_d, feed_dict={noise:n, im:real, is_training:True})
            else: # train g
                sess.run(optimizer_g, feed_dict={noise:n, is_training:True})
                train_d=True
                
            if counter==2:# every n times of training d, train g
                counter=0
                train_d=False
            counter+=1
        
            p_real=tf.reduce_mean(tf.cast(real_out>0,tf.float32))
            p_fake=1-tf.reduce_mean(tf.cast(fake_out<0,tf.float32))
            res=sess.run(
                [g_loss,d_loss,p_real,p_fake,tf.reduce_mean(log_w), tf.reduce_mean(log_g),
                 tf.reduce_mean(self.generator)],
                feed_dict={im:real, noise:n, is_training:False}) # ?????
                        
            
            self.res_dic["g_loss"][epoch]=res[0]
            self.res_dic["d_loss"][epoch]=res[1]
            self.res_dic["p_real"][epoch]=res[2]
            self.res_dic["p_fake"][epoch]=res[3]
            self.res_dic["log_w"][epoch]=res[4]
            self.res_dic["log_g"][epoch]=res[5]
            print("%d D loss: %f, p(real): %.2f%%, p(fake): %.2f%% G loss: %f, log_w: %f, log_g: %f, g_out: %f" %(
                epoch+1, res[1],res[2]*100,res[3]*100, res[0], res[4],res[5],res[6]))
            
            if epoch%200 == 0:
                print("Save all..")
                self.sample_images(epoch, image_dir, sess, noise,is_training)
                prefix="B{}_lrg{}_lrd{}".format(epoch, self.lr_g, self.lr_d)
                saver.save(sess, binary_dir+"/lrg{}_lrd{}".format(self.lr_g, self.lr_d))
                self.save_hist(binary_dir+"/lrg{}_lrd{}".format(self.lr_g, self.lr_d)+".npy")
                
            if epoch%1000==0: # pack to google drive
                print("Pack to Google drive")
#                 !zip -r /content/result_tf.zip /content/result_tf
#                 !mv /content/result_tf.zip /content/gdrive/My\ Drive/bsgan/tf
    
    def save_hist(self, path):
        arr=np.array(list(self.res_dic.values()))
        np.save(path, arr)
            
    
    def sample_images(self, epoch, image_dir, sess, noise, is_training):
        r, c = 5, 5
        n = np.random.normal(0., 1., (r * c, self.latent_dim)) # uniform random
        gen_imgs = sess.run(self.generator, feed_dict={noise:n,is_training:False})
        
#         gen_imgs = 1/(np.exp(-gen_imgs)+1) # Rescale images 0 - 1

        fig, axs = plt.subplots(r, c)
        cnt = 0
        
        prefix="B{}_lrg{}_lrd{}".format(epoch, self.lr_g, self.lr_d)
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(image_dir+"/"+prefix+".png")
        plt.close()

In [6]:
out_path=os.getcwd()+"/result_tf"
binary_dir, image_dir=setup_out_dir(out_path)
source=os.getcwd()+"/binary-mnist/original_28x28/all_digits_binary_pixels/x_train.npy"
binary_dir, image_dir, source

('/content/result_tf/binaries',
 '/content/result_tf/images',
 '/content/binary-mnist/original_28x28/all_digits_binary_pixels/x_train.npy')

In [0]:
a=Bsgan_binary()
a.build(latent_dim=100, img_shape=(28,28,1), batch_size=128)
a.train(source, binary_dir, image_dir, stepoch=0, retrain=True,
        n_samples=20,epochs=20000, sample_interval=50,lr_g=1e-4, lr_d=2e-4)

Reading MNIST, 
Total number of data 60000
Sample number of data 40000
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Generator output shape:  (?, 28, 28, 1)
Instructions for updating:
Use keras.layers.conv2d instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Discriminator output shape:  (?, 1)
Discriminator output shape:  (2560, 1)
ccc (20, 128) (20, 128) (1, 128) (20, 128)
Instructions for updating:
Use tf.cast instead.
1 D loss: 1.439364, p(real): 100.00%, p(fake): 100.00% G loss: 542.807251, log_w: 0.183767, log_g: -542.808716, g_out: 0.499995
Save all..
Pack to Google drive
  add